In [1]:
import csv
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
import skimage
from skimage.io import ImageCollection,concatenate_images,imread
from skimage.transform import resize
from skimage.color import gray2rgb
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from keras.layers import Flatten, Dense, Dropout
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from keras.applications.resnet50 import decode_predictions
#from keras.applications.xception import Xception
from keras.applications import resnet50
from keras.applications import xception
from keras.applications import vgg16
from sklearn.ensemble import VotingClassifier

#import mahotas as mt
#import xgboost as xg




Using TensorFlow backend.


In [2]:
from keras import applications
model = applications.resnet.ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

In [3]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [4]:
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from keras.applications.resnet50 import decode_predictions
path = 'dataset/train/train_8084.jpg'



In [5]:
def returnKerasImages(path):
    lab = pd.read_csv('dataset/train.csv')
    images = []
    files = []
    labels = []
    for index, row in lab.iterrows():
        files.append(row['Id'])
        labels.append(row['Category'])
#        print()
    print(files[0])
    target = []
    for i in range(len(files)):
#        print(i)
        name = path+'/'+files[i]
        overallPath = path+'train/'+files[i]
 #       print(overallPath)
    
        img = image.load_img(overallPath, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = resnet50.preprocess_input(x)

        images.append(x)
        
        target.append(labels[i])
        
    return images,target
        
        




In [6]:
keras_images, target = returnKerasImages('dataset/')

train_0.jpg


In [7]:
#input a small number of n here to test the methodology because n = 10,000 will take forever on CPU. 
# n = 500, gives 65% accuracy. 
# here n is the number of samples. 
n = 500
def resnet_features(keras_images):
    res_features = []
    for i in range((n)):
        res_features.append(model.predict(keras_images[i]))
    return res_features
        
    

In [9]:
res_feats = resnet_features(keras_images)


res_feats = np.array(res_feats).reshape(n,1000)

X_train, X_test, y_train, y_test = train_test_split(res_feats, target[0:n], test_size=0.20, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=20, random_state=42)

from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(n_estimators=500,max_features='sqrt',n_jobs=-1,random_state=10)

rf.fit(X_train, y_train)
print("Train accuracy: ", rf.score(X_train,y_train))
print("Test accuracy: ", rf.score(X_test,y_test))







Train accuracy:  1.0
Test accuracy:  0.56
